In [ ]:
from datetime import datetime
import random
from math import radians, sin, cos, sqrt, atan2

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2): #거리 계산 해주는
    R = 6371  # 지구의 반경(km)

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c

    return distance

In [ ]:
# 우선순위를 기반으로 환자 리스트 생성
def route(doctor_location, patients_data, additional_calls):
    # 5km 반경 내 환자 필터링
    nearby_patients = []
    for patient in patients_data:
        distance = calculate_distance(
            doctor_location[0], doctor_location[1],
            patient['location'][0], patient['location'][1]
        )
        if distance <= 5:
            nearby_patients.append(patient)

    # 추가호출 환자가 반드시 포함되도록 처리
    selected_patients = []
    additional_patients = [p for p in nearby_patients if p['id'] in additional_calls]
    regular_patients = [p for p in nearby_patients if p['id'] not in additional_calls]

    # 랜덤하게 5~7명 선택
    current_date = datetime.now()
    random.seed(current_date.year * 10000 + current_date.month * 100 + current_date.day)
    global total_patients_needed
    total_patients_needed = random.randint(4, 8)

    # 추가호출 환자를 먼저 포함
    selected_patients.extend(additional_patients)

    # 남은 자리를 일반 환자로 채움
    remaining_slots = total_patients_needed - len(additional_patients)
    if remaining_slots > 0:
        selected_patients.extend(regular_patients[:remaining_slots])

    return selected_patients

In [ ]:
# 총 거리 계산 함수
def calculate_total_distance(route_plan, doctor_location):
    total_distance = 0.0
    current_location = doctor_location
    # 경로 상의 모든 환자 사이 거리 합산
    for patient in route_plan:
        next_location = patient['location']
        total_distance += calculate_distance(
            current_location[0], current_location[1],
            next_location[0], next_location[1]
        )
        current_location = next_location

    return total_distance

In [ ]:
# route함수로 생성한 환자들 사이 최단 경로 재배열 (총 이동 거리를 최소화)
def optimize_route(route_plan, doctor_location, additional_calls):
    if not route_plan:
        return []

    # 환자들의 우선순위 점수 계산 함수
    def get_priority_score(patient):
        base_score = 0
        # 진료일 경과된 환자 (가장 높은 우선순위)
        if patient['remaining_days'] < 0:
            base_score = 1000 - patient['remaining_days']  # 더 오래 경과될수록 높은 점수
        # 진료예정 추가요청 환자 (중간 우선순위)
        elif patient['id'] in additional_calls:
            base_score = 500 + (10 - patient['remaining_days'])  # 예정일이 가까울수록 높은 점수
        # 일반 진료예정 환자 (낮은 우선순위)
        else:
            base_score = 10 - patient['remaining_days']  # 예정일이 가까울수록 높은 점수
        return base_score

    # 최적 경로 찾기
    current_location = doctor_location
    remaining_patients = route_plan.copy()
    final_route = []

    while remaining_patients:
        best_next_patient = None
        best_score = float('-inf')

        for patient in remaining_patients:
            # 거리 계산 (가까울수록 높은 점수)
            distance = calculate_distance(
                current_location[0], current_location[1],
                patient['location'][0], patient['location'][1]
            )
            distance_score = 1.0 / (distance + 0.1)  # 거리가 가까울수록 높은 점수 (0으로 나누기 방지)

            # 우선순위 점수
            priority_score = get_priority_score(patient)

            # 종합 점수 (우선순위와 거리를 모두 고려)
            # 우선순위에 더 큰 가중치 부여
            total_score = (priority_score * 10) + (distance_score * 5)

            if total_score > best_score:
                best_score = total_score
                best_next_patient = patient

        if best_next_patient:
            final_route.append(best_next_patient)
            remaining_patients.remove(best_next_patient)
            current_location = best_next_patient['location']

    return final_route

In [ ]:
#각 환자 사이의 거리 계산 함수
def calc_route_distance(route_plan, doctor_location):
    if not route_plan:  # 경로가 비어있는 경우
        return []

    route_distance = []

    # 의사와 첫번째 환자 사이의 거리 계산
    route_distance.append(calculate_distance(
            doctor_location[0], doctor_location[1],
            route_plan[0]['location'][0], route_plan[0]['location'][1]
        ))

    # 첫번째 환자부터 각 환자 사이의 거리를 계산하여 추가
    for i in range(len(route_plan)-1):
        current_location = route_plan[i]['location']
        next_location = route_plan[i+1]['location']
        distance = calculate_distance(
            current_location[0], current_location[1],
            next_location[0], next_location[1]
        )
        route_distance.append(distance)

    return route_distance

In [ ]:
# 테스트를 위한 메인 부분 추가
if __name__ == "__main__":
    # 의사 현재 위치 (서울시청 좌표)
    doctor_loc = [37.2636, 127.0286]
    patients = [
    {
            'id': 1,
            'location': [37.2650, 127.0300],  # 시청 근처
            'remaining_days': -2  # 2일 지난 환자
        },
        {
            'id': 2,
            'location': [37.2620, 127.0270],  # 시청 근처
            'remaining_days': -5  # 5일 지난 환자
        },
        {
            'id': 3,
            'location': [37.2645, 127.0320],  # 시청 근처
            'remaining_days': 3   # 3일 후 진료 예정
        },
        {
            'id': 4,
            'location': [37.2615, 127.0260],  # 시청 근처
            'remaining_days': -1  # 1일 지난 환자
        },
        {
            'id': 5,
            'location': [37.2670, 127.0290],  # 시청 근처
            'remaining_days': 1   # 1일 후 진료 예정
        },
        {
            'id': 6,
            'location': [37.2600, 127.0250],  # 시청 근처
            'remaining_days': -7  # 7일 지난 환자
        },
        {
            'id': 7,
            'location': [37.2660, 127.0350],  # 시청 근처
            'remaining_days': -3  # 3일 지난 환자
        },
        {
            'id': 8,
            'location': [37.2585, 127.0235],  # 시청 근처
            'remaining_days': 2   # 2일 후 진료 예정
        },
        {
            'id': 9,
            'location': [37.2700, 127.0370],  # 시청 근처
            'remaining_days': -4  # 4일 지난 환자
        },
        {
            'id': 10,
            'location': [37.2680, 127.0330],  # 시청 근처
            'remaining_days': 4   # 4일 후 진료 예정
        },
        {
            'id': 11,
            'location': [37.2590, 127.0220],  # 시청 근처
            'remaining_days': -6  # 6일 지난 환자
        },
        {
            'id': 12,
            'location': [37.2710, 127.0385],  # 시청 근처
            'remaining_days': 5   # 5일 후 진료 예정
        },
        {
            'id': 13,
            'location': [37.2575, 127.0205],  # 시청 근처
            'remaining_days': -8  # 8일 지난 환자
        },
        {
            'id': 14,
            'location': [37.2630, 127.0360],  # 시청 근처
            'remaining_days': 0   # 당일 진료 예정
        },
        {
            'id': 15,
            'location': [37.2655, 127.02559],  # 시청 근처
            'remaining_days': -9  # 9일 지난 환자
        }
]

    additional_calls = [3, 11]

    # route 함수 호출 및 결과 출력
    route_plan = route(doctor_loc, patients, additional_calls)
    best_route = optimize_route(route_plan, doctor_loc, additional_calls)
    route_distance = calc_route_distance(best_route, doctor_loc)

    print("\n=== 방문 계획 ===")
    print(f"총 방문 환자 수: {len(best_route)}명")
    print("\n[방문 순서]")
    for i, patient in enumerate(best_route, 1):
        print(f"{i}번째 방문: 환자 ID {patient['id']}")
        print(f"  ㄴ 진료 예정일 기준: {abs(patient['remaining_days'])}일 {'경과' if patient['remaining_days'] < 0 else '전'}")
        print(f"  ㄴ 위치: {patient['location']}")

    print("\n[이동 거리]")
    for i, distance in enumerate(route_distance):
        if i == 0:
            print(f"• 의사 → 1번째 환자: {int(distance*1000):,}m")
        else:
            print(f"• {i}번째 환자 → {i+1}번째 환자: {int(distance*1000):,}m")

    print("\n[환자 정보 요약]")
    route_summary = [
        {
            "ID": patient['id'],
            "remaining_days": patient['remaining_days'],
            "location": patient['location']
        }
        for patient in best_route
    ]
    for patient_info in route_summary:
        print(patient_info)


=== 방문 계획 ===
총 방문 환자 수: 6명

[방문 순서]
1번째 방문: 환자 ID 11
  ㄴ 진료 예정일 기준: 6일 경과
  ㄴ 위치: [37.259, 127.022]
2번째 방문: 환자 ID 2
  ㄴ 진료 예정일 기준: 5일 경과
  ㄴ 위치: [37.262, 127.027]
3번째 방문: 환자 ID 4
  ㄴ 진료 예정일 기준: 1일 경과
  ㄴ 위치: [37.2615, 127.026]
4번째 방문: 환자 ID 1
  ㄴ 진료 예정일 기준: 2일 경과
  ㄴ 위치: [37.265, 127.03]
5번째 방문: 환자 ID 3
  ㄴ 진료 예정일 기준: 3일 전
  ㄴ 위치: [37.2645, 127.032]
6번째 방문: 환자 ID 5
  ㄴ 진료 예정일 기준: 1일 전
  ㄴ 위치: [37.267, 127.029]

[이동 거리]
• 의사 → 1번째 환자: 776m
• 1번째 환자 → 2번째 환자: 554m
• 2번째 환자 → 3번째 환자: 104m
• 3번째 환자 → 4번째 환자: 526m
• 4번째 환자 → 5번째 환자: 185m
• 5번째 환자 → 6번째 환자: 384m

[환자 정보 요약]
{'ID': 11, 'remaining_days': -6, 'location': [37.259, 127.022]}
{'ID': 2, 'remaining_days': -5, 'location': [37.262, 127.027]}
{'ID': 4, 'remaining_days': -1, 'location': [37.2615, 127.026]}
{'ID': 1, 'remaining_days': -2, 'location': [37.265, 127.03]}
{'ID': 3, 'remaining_days': 3, 'location': [37.2645, 127.032]}
{'ID': 5, 'remaining_days': 1, 'location': [37.267, 127.029]}


In [ ]:
!pip install fastapi uvicorn


In [ ]:
!ngrok config add-authtoken "EA6NHGIRTLM6T7KRW2LLFDTOZCHSHFJP"

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from google.colab import output
import asyncio

# FastAPI 앱 생성
app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 데이터 모델 정의
class PatientData(BaseModel):
    patients: list
    doctor_location: list
    additional_calls: list

# 기본 경로 추가
@app.get("/")
async def root():
    return {"message": "서버가 정상적으로 실행 중입니다"}

# 데이터 처리 엔드포인트
@app.post("/process_data")
async def process_data(data: PatientData):
    try:
        result = create_route_plan(data.patients, data.doctor_location, data.additional_calls)
        return {"route": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# 서버 실행을 위한 함수
async def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

# Colab에서 실행
from IPython.display import clear_output
import threading

def run_fastapi():
    asyncio.run(run_server())

# 새로운 스레드에서 서버 실행
thread = threading.Thread(target=run_fastapi, daemon=True)
thread.start()

# URL 출력
print("서버가 시작되었습니다.")
output.serve_kernel_port_as_window(8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
서버가 시작되었습니다.
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

INFO:     Started server process [16511]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!pip install folium

# 필요한 라이브러리 임포트
import folium
from folium import plugins

In [ ]:


# 지도 초기화 (서울 중심)
m = folium.Map(location=[37.2636, 127.0286], zoom_start=14)

# 마커 위치 리스트 (선 그리기 위해)
locations = []

patients = [
{'ID': 11, 'remaining_days': -6, 'location': [37.259, 127.022]},
{'ID': 2, 'remaining_days': -5, 'location': [37.262, 127.027]},
{'ID': 4, 'remaining_days': -1, 'location': [37.2615, 127.026]},
{'ID': 1, 'remaining_days': -2, 'location': [37.265, 127.03]},
{'ID': 3, 'remaining_days': 3, 'location': [37.2645, 127.032]},
{'ID': 5, 'remaining_days': 1, 'location': [37.267, 127.029]}
]


# 방문 순서대로 마커 추가
for patient in patients:
    folium.Marker(
        location=patient['location'],  # 위치
        popup=f"ID: {patient['ID']}<br>Remaining Days: {patient['remaining_days']}",  # 팝업 내용
        icon=folium.Icon(color='blue', icon='info-sign')  # 마커 아이콘
    ).add_to(m)

    # 위치 리스트에 추가 (마커들 연결을 위해)
    locations.append(patient['location'])

# 마커들 이어주는 선 추가
folium.PolyLine(locations, color='blue', weight=2.5, opacity=1).add_to(m)


In [ ]:
m